In [1]:
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

import pandas as pd



In [2]:
connection = DatabaseConnector()
engine = connection.init_db_engine()

extractor = DataExtractor()
df = extractor.read_rds_table(engine, "legacy_users")

In [ ]:
# class DataCleaning():
  
#   def clean_user_data(self, pandas_dataframe):
#     df = pandas_dataframe

#     # df.drop("first_name", axis=1, inplace=True)

#     return cleaned df




Checking for missing/null values in the columns

In [7]:

df.isnull().sum()

index            0
first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64

The above shows that there are no missing or null values in any column.

We can check now that the index is unique.

In [9]:
is_unique = df["index"].is_unique
is_unique

True

In [10]:
is_unique = df["user_uuid"].is_unique
is_unique

False

In [11]:
duplicate_uuids = df[df.duplicated("user_uuid", keep=False)]
duplicate_uuids

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
866,867,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1022,1023,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1805,1807,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2103,2103,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2437,2439,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2739,6526,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2764,2764,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4984,4987,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
5307,5310,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
6920,6927,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
